In [1]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("mySpark").getOrCreate()

24/07/08 09:25:54 WARN Utils: Your hostname, Nombas-MacBook-Pro-2.local resolves to a loopback address: 127.0.0.1; using 192.168.0.3 instead (on interface en0)
24/07/08 09:25:54 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/07/08 09:25:55 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
flightData = spark\
    .read.option("inferSchema", "true")\
    .option("header", "true")\
    .csv("Dataset_salary.csv")

In [3]:
flightData.take(3)

[Row(experience_level='SE', job_title='AI Engineer', salary=202730, company_size='M', employee_residence='US', remote_ratio=90),
 Row(experience_level='SE', job_title='AI Engineer', salary=92118, company_size='M', employee_residence='US', remote_ratio=40),
 Row(experience_level='SE', job_title='Data Engineer', salary=130500, company_size='M', employee_residence='US', remote_ratio=36)]

In [7]:
flightData.sort("job_title").explain()

== Physical Plan ==
AdaptiveSparkPlan isFinalPlan=false
+- Sort [job_title#18 ASC NULLS FIRST], true, 0
   +- Exchange rangepartitioning(job_title#18 ASC NULLS FIRST, 200), ENSURE_REQUIREMENTS, [plan_id=49]
      +- FileScan csv [experience_level#17,job_title#18,salary#19,company_size#20,employee_residence#21,remote_ratio#22] Batched: false, DataFilters: [], Format: CSV, Location: InMemoryFileIndex(1 paths)[file:/Users/nombauser/Desktop/GIT/my_git_repos/LearningSpark/Dataset_s..., PartitionFilters: [], PushedFilters: [], ReadSchema: struct<experience_level:string,job_title:string,salary:int,company_size:string,employee_residence...




In [8]:
spark.conf.set("spark.sql.shuffle.partitions", "5")
flightData.sort("remote_ratio").take(2)

[Row(experience_level='MI', job_title='Data Engineer', salary=60300, company_size='M', employee_residence='US', remote_ratio=11),
 Row(experience_level='MI', job_title='Data Visualization Specialist', salary=67750, company_size='M', employee_residence='US', remote_ratio=11)]

In [9]:
flightData.printSchema()

root
 |-- experience_level: string (nullable = true)
 |-- job_title: string (nullable = true)
 |-- salary: integer (nullable = true)
 |-- company_size: string (nullable = true)
 |-- employee_residence: string (nullable = true)
 |-- remote_ratio: integer (nullable = true)



In [4]:
flightData.show(4)

+----------------+-------------+------+------------+------------------+------------+
|experience_level|    job_title|salary|company_size|employee_residence|remote_ratio|
+----------------+-------------+------+------------+------------------+------------+
|              SE|  AI Engineer|202730|           M|                US|          90|
|              SE|  AI Engineer| 92118|           M|                US|          40|
|              SE|Data Engineer|130500|           M|                US|          36|
|              SE|Data Engineer| 96000|           M|                US|          55|
+----------------+-------------+------+------------+------------------+------------+
only showing top 4 rows



#### Creating a Temporary View

In [6]:
flightData.createOrReplaceTempView("Dataset")

##### Let's do it in two ways

In [7]:
sqlway = spark.sql(
"""
    SELECT
        job_title,
        COUNT(employee_residence)
    FROM 
    Dataset
    GROUP BY 1
    """)

sqlway.show()

sqlway.explain()

+--------------------+-------------------------+
|           job_title|count(employee_residence)|
+--------------------+-------------------------+
|        Data Modeler|                        2|
|Data Visualizatio...|                        2|
|Data Operations M...|                        2|
|Data Integration ...|                        2|
|Machine Learning ...|                        8|
|Business Intellig...|                        2|
|  Research Scientist|                        6|
|      Data Scientist|                       15|
|        Data Analyst|                       22|
|         ML Engineer|                        4|
|   Applied Scientist|                        2|
|Data Product Manager|                        2|
|        Data Manager|                        4|
|Data Science Manager|                        2|
|        NLP Engineer|                        2|
|      Data Architect|                        4|
|         AI Engineer|                        2|
|Business Intellig..

In [38]:
dataFrame_way = flightData\
    .groupBy("job_title")\
    .count()

dataFrame_way.explain()

== Physical Plan ==
AdaptiveSparkPlan isFinalPlan=false
+- HashAggregate(keys=[job_title#89], functions=[count(1)])
   +- Exchange hashpartitioning(job_title#89, 5), ENSURE_REQUIREMENTS, [plan_id=357]
      +- HashAggregate(keys=[job_title#89], functions=[partial_count(1)])
         +- FileScan csv [job_title#89] Batched: false, DataFilters: [], Format: CSV, Location: InMemoryFileIndex(1 paths)[file:/Users/nombauser/Desktop/GIT/my_git_repos/LearningSpark/Dataset_s..., PartitionFilters: [], PushedFilters: [], ReadSchema: struct<job_title:string>




In [40]:
# Importing functions

from pyspark.sql.functions import *
flightData.select(max("salary")).take(2)

[Row(max(salary)=400000)]

In [13]:
from pyspark.sql import *
ss = spark.sql(
"""
    SELECT MAX(SALARY) AS `Highest Salary`
    FROM Dataset
""")

ss.show()
ss.explain()

+--------------+
|Highest Salary|
+--------------+
|        400000|
+--------------+

== Physical Plan ==
AdaptiveSparkPlan isFinalPlan=false
+- HashAggregate(keys=[], functions=[max(SALARY#19)])
   +- Exchange SinglePartition, ENSURE_REQUIREMENTS, [plan_id=169]
      +- HashAggregate(keys=[], functions=[partial_max(SALARY#19)])
         +- FileScan csv [salary#19] Batched: false, DataFilters: [], Format: CSV, Location: InMemoryFileIndex(1 paths)[file:/Users/nombauser/Desktop/GIT/my_git_repos/LearningSpark/Dataset_s..., PartitionFilters: [], PushedFilters: [], ReadSchema: struct<salary:int>




#### Writing a more complex Transfromation with our DataSet

In [8]:
maxSql = spark.sql("""
    SELECT 
        job_title,
        MAX(salary)
    FROM Dataset
    GROUP BY 1
    ORDER BY 2 DESC
    LIMIT 5
""")

# maxSql.collect()
maxSql.show()

+--------------------+-----------+
|           job_title|max(salary)|
+--------------------+-----------+
|         ML Engineer|     400000|
|  Research Scientist|     260570|
|      Data Scientist|     257600|
|Machine Learning ...|     251250|
|Data Science Manager|     240000|
+--------------------+-----------+



In [9]:
from pyspark.sql.functions import *

df = flightData.groupBy("job_title")\
    .max("salary")\
    .withColumnRenamed("max(salary)", "Maximum Salary")\
    .sort(desc("Maximum Salary"))\
    .limit(5)

df.show()

+--------------------+--------------+
|           job_title|Maximum Salary|
+--------------------+--------------+
|         ML Engineer|        400000|
|  Research Scientist|        260570|
|      Data Scientist|        257600|
|Machine Learning ...|        251250|
|Data Science Manager|        240000|
+--------------------+--------------+

